In [21]:
from sparg import process_trees, dispersal, locate
import msprime
import numpy as np
import os, csv

# Estimating dispersal

## A simple check

We first generate a tree sequence with n samples using msprime

In [35]:
n = 100
Ne = 5000
L = 1e8
RBP = 5e-9
U = 1.25e-8
ts = msprime.simulate(sample_size=n, Ne=Ne, length=L, recombination_rate=RBP, mutation_rate=U, random_seed=1)
ts

Now let's use sparg to process some evenly spaced trees 

In [36]:
ntrees = 100 #number of trees to process
which_trees = [int(i) for i in np.linspace(0, ts.num_trees-1, ntrees)] #evenly space these trees along the tree sequence

shared_times, samples = process_trees.process_trees(ts=ts, which_trees=which_trees, from_ts=True, important=False) #use sparg to process the trees

converting to dendropy trees...


  0%|          | 0/100 [00:00<?, ?it/s]

processing...


  0%|          | 0/100 [00:00<?, ?it/s]

Now, to do a simple check, we'll position the samples as if they were located based on Brownian motion down the first tree with variance sigma^2 in x and y and no covariance

In [37]:
sigma = 0.5
np.random.seed(1)
xs = np.random.multivariate_normal(mean = np.zeros(n), cov = sigma**2 * shared_times[0][0][0])
ys = np.random.multivariate_normal(mean = np.zeros(n), cov = sigma**2 * shared_times[0][0][0])
ix = [np.where(np.array(samples[0][0][0]) == i)[0][0] for i in range(n)] #clunky way to get order of samples
locations = np.array([[xs[i], ys[i]] for i in ix])

And then we can use sparg to estimate dispersal at this tree and check that our method estimates something close to the true dispersal rate

In [38]:
x0 = [0.5, 0.5, 0] #initial guess for parameters (sd in x, sd in y, correlation bw x and y)
bnds = ((1e-6,None), (1e-6,None), (-0.99,0.99)) # bounds on the parameters (eg to keep the sd's positive)

mle = dispersal.estimate(locations=locations, shared_times=shared_times[:1], samples=samples[:1], important=False, x0=x0, bnds=bnds)

searching for maximum likelihood parameters...
the max is  [0.44363001 0.47345042 0.09404812]
finding the max took 0.2248096466064453 seconds


Good, the SD in x and y (the first two numbers, respectively) are both close to sigma and the correlation (the final number) is close to 0.

In this case we can actually just calculate the mle analytically, as a check that the numerical optimizer is working

In [39]:
Sigma = dispersal.mle(locations=locations[samples[0][0][0]], shared_time=shared_times[0][0][0]) #here we use the first subtree of the first sample at the first locus
Sigma[0,0]**0.5, Sigma[1,1]**0.5, Sigma[0,1] / (Sigma[0,0]**0.5 * Sigma[1,1]**0.5) #convert from covariance matrix to SDs and correlation

(0.44363002150525394, 0.4734504767382652, 0.09404789956492444)

In practice we'll want to find the maximum composite likelihood estimate of dispersal rate across many trees

In [7]:
mcle = dispersal.estimate(locations=locations, shared_times=shared_times, samples=samples, important=False, x0=x0, bnds=bnds) #might take ~2 minutes

searching for maximum likelihood parameters...
the max is  [ 9.80585519  7.60829136 -0.89567912]
finding the max took 73.14066767692566 seconds


The dispersal rate will, in general, be higher when we use trees other than the first, which is the only tree we determined the locations by.

## Using inferred trees

In practice we will not know the true trees and so will want to infer the trees and then integrate over uncertainty by sampling each tree multiple times. 

In [40]:
datadir = 'tutorial_data/'
filename = 'example'
fname = datadir + filename
PATH_TO_RELATE = '~/programs/relate_v1.1.4_x86_64_static/' #update this for your machine (download from https://myersgroup.github.io/relate/index.html)
memory = 5 #Gb

# create vcf from tree sequence
with open(datadir + filename + ".vcf.gz", "w") as vcf_file:
    ts.write_vcf(vcf_file, ploidy=2)

# convert vcf to haps/sample format
script="%s/bin/RelateFileFormats \
            --mode ConvertFromVcf \
            --haps %s.haps \
            --sample %s.sample \
            --chr 1 \
            -i %s" %(PATH_TO_RELATE, fname, fname, fname)
os.system(script)

# # overwrite sample file with haploid version
# with open(fname + '.sample', mode='w') as file:
#     writer = csv.writer(file, delimiter=' ') #space delimited
#     writer.writerow(['ID_1','ID_2','missing']) #header
#     writer.writerow(['0','0','0']) #for some reason relate starts with this null first row, https://myersgroup.github.io/relate/input_data.html
#     for i in range(n):
#         data = np.hstack((i, 'NA', 0)) #add each individual as row, with NA signifying that these are haploids
#         writer.writerow(data)
# file.close()

# make uniform recombination map
r = (1 - (1 - 2 * RBP)**L)/2 #recombination distance from one end of chromosome to other
cm = 50 * np.log(1/(1-2*r)) #length in centiMorgans
cr = cm/L * 1e6 #cM per million bases
script = "pos COMBINED_rate Genetic_Map \n0 %f 0 \n%d %f %f" %(cr, L, cr, cm)
os.system("echo '" + script + "' > %s.map" %fname)

# infer tree with Relate (using correct mutation rate and recombination map, and estimated (diploid) effective population size)
script="%s/bin/Relate \
            --mode All \
            -m %.10f \
            -N %f \
            --haps %s.haps \
            --sample %s.sample \
            --map %s.map \
            --memory %d \
            -o %s" %(PATH_TO_RELATE, U, 2*Ne, fname, fname, fname, memory, filename)
os.system(script)

256

# Locating ancestors

In [19]:
'%.10f' %U

'0.0000000125'